In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import numpy as np
import pandas as pd
from lxml import etree
import re

In [2]:
def scrape_table(table):
    data = []
    for row in table.find_all("tr")[1:]:  # Skip the header row
        cells = row.find_all("td")
        row_data = [cell.text.strip() for cell in cells]
        data.append(row_data)
    return data

In [11]:
### S&P 500 ###
'''
Code deprecated, use selenium
'''

# url = "https://stockanalysis.com/list/sp-500-stocks/"
# response = requests.get(url)
# html_content = response.content
# soup = BeautifulSoup(html_content, "html.parser")
# table = soup.find("table", {"id":"main-table"})
# sp500_data = scrape_table(table)

In [3]:
### Code for test scraping with S&P500 ###
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--headless")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://stockanalysis.com/list/sp-500-stocks/")

columns_dropdown_button = driver.find_element(By.XPATH, "/html/body/div/div[1]/div[2]/main/div/div/div/div[3]/div[2]/div[3]/button")
columns_dropdown_button.click()
sector_checkbox_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div/div[1]/div[2]/main/div/div/div/div[3]/div[2]/div[3]/div/div[2]/div[10]/input"))
)
sector_checkbox_button.click()
time.sleep(3)
soup = BeautifulSoup(driver.page_source, "html.parser")
table = soup.find("table", {"id":"main-table"})
sp500_data = scrape_table(table)

,0,1,2,3,4,5,6,7
0,1,MSFT,Microsoft Corporation,"3,179.71B",427.93,1.10%,227.58B,Technology
1,2,AAPL,Apple Inc,"2,702.95B",175.04,4.33%,385.71B,Technology
2,3,NVDA,NVIDIA Corporation,"2,265.40B",906.16,4.11%,60.92B,Technology
3,4,GOOGL,Alphabet Inc.,"1,990.74B",159.41,2.09%,307.39B,Communication Services
4,5,GOOG,Alphabet Inc.,"1,988.62B",160.79,1.99%,307.39B,Communication Services
...,...,...,...,...,...,...,...,...
498,499,MHK,"Mohawk Industries, Inc.",7.50B,117.68,-1.66%,11.14B,Consumer Discretionary
499,500,IVZ,Invesco Ltd.,7.06B,15.71,-1.01%,5.72B,Financials
500,501,PARA,Paramount Global,7.03B,11.27,7.33%,29.65B,Communication Services
501,502,CMA,Comerica Incorporated,6.85B,51.68,0.56%,3.50B,Financials


In [12]:
sp500df = pd.DataFrame(sp500_data, columns=["No.", "Symbol", "Company Name", "Market Cap", "Stock Price", "% Change", "Revenue", "Sector"], )

In [3]:
#### NYSE STOCKS ####
data = []
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--headless")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://stockanalysis.com/list/nyse-stocks/")
for i in range(4):
    soup = BeautifulSoup(driver.page_source, "html.parser")
    table = soup.find("table", {"id":"main-table"})
    data.extend(scrape_table(table))
    next_button = driver.find_element(By.XPATH, "/html/body/div/div[1]/div[2]/main/div/div/div/nav/button[2]")
    next_button.click()
    time.sleep(3)

In [4]:
df = pd.DataFrame(data, columns=["No.", "Symbol", "Company Name", "Market Cap", "Stock Price", "% Change", "Revenue"], )

In [5]:
df

,No.,Symbol,Company Name,Market Cap,Stock Price,% Change,Revenue
0,1,BRK.A,Berkshire Hathaway Inc.,885.65B,"618,950.00",-1.27%,439.34B
1,2,BRK.B,Berkshire Hathaway Inc.,884.96B,409.11,-1.35%,439.34B
2,3,LLY,Eli Lilly and Company,724.46B,761.98,0.63%,34.12B
3,4,TSM,Taiwan Semiconductor Manufacturing Company Lim...,636.30B,146.22,0.56%,70.35B
4,5,JPM,JPMorgan Chase & Co.,563.03B,195.47,-0.85%,158.10B
...,...,...,...,...,...,...,...
1975,1976,FATH,Fathom Digital Manufacturing Corporation,16.16M,4.59,1.66%,139.34M
1976,1977,NYC,American Strategic Investment Co,16.06M,6.13,-8.23%,62.71M
1977,1978,LEJU,Leju Holdings Limited,15.16M,1.10,-3.51%,332.27M
1978,1979,XIN,"Xinyuan Real Estate Co., Ltd.",14.68M,2.70,-1.64%,754.13M


In [6]:
top_100_stock_ticker = df.iloc[:100].Symbol

In [16]:
top_100_sp500_stock_ticker = sp500df.iloc[:100].Symbol

In [17]:
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--headless")
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(options=chrome_options)
smart_score_results = []

for stock_ticker in top_100_sp500_stock_ticker:
    try:
        driver.get(f"https://www.tipranks.com/stocks/{stock_ticker}")
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[2]/div[4]"))
        )
        smart_score = re.search(r"Stock Smart Score\s*(\d+)", element.text).group(1)
        smart_score_results.append((stock_ticker, smart_score))
    except:
        print(f"Error printing smart score for {stock_ticker}")
        continue
    
    #print(f"{stock_ticker} smart score: {smart_score}")

Error printing smart score for ACN
Error printing smart score for PBR.A


In [19]:
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--headless")
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(options=chrome_options)
smart_score_results = []

driver.get(f"https://www.tipranks.com/stocks/V")
element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[2]/div[4]"))
)
smart_score = re.search(r"Stock Smart Score\s*(\d+)", element.text).group(1)
smart_score

'9'

In [18]:
sorted(smart_score_results, key=lambda x: int(x[1]), reverse=True)

[('TSM', '10'),
 ('WMT', '10'),
 ('XOM', '10'),
 ('MA', '10'),
 ('ORCL', '10'),
 ('TM', '10'),
 ('CVX', '10'),
 ('CRM', '10'),
 ('KO', '10'),
 ('SHEL', '10'),
 ('SAP', '10'),
 ('DIS', '10'),
 ('IBM', '10'),
 ('BX', '10'),
 ('RTX', '10'),
 ('UPS', '10'),
 ('T', '10'),
 ('C', '10'),
 ('LLY', '9'),
 ('NVO', '9'),
 ('V', '9'),
 ('MRK', '9'),
 ('ABBV', '9'),
 ('BAC', '9'),
 ('TMO', '9'),
 ('CAT', '9'),
 ('GE', '9'),
 ('TTE', '9'),
 ('PFE', '9'),
 ('HDB', '9'),
 ('NKE', '9'),
 ('SYK', '9'),
 ('NEE', '9'),
 ('ETN', '9'),
 ('PGR', '9'),
 ('UL', '9'),
 ('ELV', '9'),
 ('DE', '9'),
 ('RIO', '9'),
 ('BP', '9'),
 ('CI', '9'),
 ('BMY', '9'),
 ('BSX', '9'),
 ('IBN', '9'),
 ('ANET', '9'),
 ('CNQ', '9'),
 ('HCA', '9'),
 ('ABT', '8'),
 ('BABA', '8'),
 ('HSBC', '8'),
 ('NOW', '8'),
 ('UBER', '8'),
 ('BHP', '8'),
 ('UNP', '8'),
 ('SPGI', '8'),
 ('SCHW', '8'),
 ('MUFG', '8'),
 ('BLK', '8'),
 ('MDT', '8'),
 ('TJX', '8'),
 ('UBS', '8'),
 ('CVS', '8'),
 ('KKR', '8'),
 ('CNI', '8'),
 ('WM', '8'),
 ('PG', '7'),